In [1]:
import sys
sys.path.append('../')
sys.path.append('../../')
from src.posterior_networks.PosteriorNetwork import PosteriorNetwork
import json
import numpy as np
import pandas as pd
import wandb
from dataset_manager.ClassificationDataset import MapillaryDataset
from src.results_manager.metrics_prior import confidence, brier_score, anomaly_detection
import torch
import torchvision.transforms as transforms
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
from src.posterior_networks.config import config
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv('/lab/project-1/train_label.csv')
val_df = pd.read_csv('/lab/project-1/val_label.csv')
test_df = pd.read_csv('/lab/project-1/test_label.csv')
regions = set(['g1', 'g2', 'g3', 'g4', 'g5', 'g6'])
classes = set(['regu', 'warn', 'comp', 'info'])
full_path = '/lab/project-1/models/effnets_oodg5_lat8_reg1e4_dens6_batch64_lr1e4_'

config = json.load(open(f'{full_path}/config.json'))

# full_df = pd.concat([train_df, val_df, test_df])
full_df = pd.concat([test_df])

if 'class_encoding' in config: 
    class_encoding = config['class_encoding']
else:
    class_encoding = {c: i for i, c in enumerate(sorted(train_df.label.unique()))}

full_df['label_encoded'] = full_df.label.map(class_encoding)
if "N" in config:
    N = config['N']
else:
    N = train_df.label_encoded.value_counts().sort_index().values
N = torch.tensor(N)

print(N)

model = PosteriorNetwork(N=N,
                         n_classes=config['num_classes'],
                         hidden_dims=config['hidden_dims'],
                         kernel_dim=None,
                         latent_dim=config['latent_dim'],
                         architecture=config['architecture'],
                         k_lipschitz=config['k_lipschitz'],
                         no_density=config['no_density'],
                         density_type=config['density_type'],
                         n_density=config['n_density'],
                         budget_function=config['budget_function'],
                         batch_size=config['batch_size'],
                         lr=config['lr'],
                         loss=config['loss'],
                         dropout=config['dropout'],
                         regr=config['regr'],
                         seed=config['seed'])

# model.load_state_dict(torch.load(f'{full_path}/best_model.pth')['model_state_dict'])
model.cuda()
transform_val_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
full_dataset = MapillaryDataset(full_df, transform = transform_val_test)

# use a dict to map ground truth vector of ID and OOD
ood_regions_classes = set(config['ood_regions'].split(','))
ood_regions = ood_regions_classes.intersection(regions)
ood_classes = ood_regions_classes.intersection(classes)

grd_truth = torch.tensor(full_df.region.isin(ood_regions).astype(int).values) # 0 as ID, 1 as OOD

tensor([ 7134,  3854, 22059, 10828])


Using cache found in /nfs/homedirs/zhz/.cache/torch/hub/pytorch_vision_main


In [3]:
# if error in model loading:
dict = torch.load(f'{full_path}/best_model.pth')['model_state_dict']
older_val = dict['sequential.4.weight']
dict['sequential.5.weight'] = dict.pop('sequential.4.weight')
older_val = dict['sequential.4.bias']
dict['sequential.5.bias'] = dict.pop('sequential.4.bias')
torch.save(full_path,'./model_changed.pth')

In [4]:
model.load_state_dict(dict)

<All keys matched successfully>

In [5]:
full_loader = torch.utils.data.DataLoader(full_dataset,
                                                      batch_size=8,
                                                      num_workers=6, pin_memory=True)
model.eval()

def compute_X_Y_alpha(model, loader, alpha_only=False):
    model.cpu()
    for batch_index, (X, Y) in enumerate(loader):
        X, Y = X.to('cpu'), Y.to('cpu')
        alpha_pred = model(X, None, return_output='alpha', compute_loss=False)
        if batch_index == 0:
            X_duplicate_all = X.to("cpu")
            orig_Y_all = Y.to("cpu")
            alpha_pred_all = alpha_pred.to("cpu")
        else:
            X_duplicate_all = torch.cat([X_duplicate_all, X.to("cpu")], dim=0)
            orig_Y_all = torch.cat([orig_Y_all, Y.to("cpu")], dim=0)
            alpha_pred_all = torch.cat([alpha_pred_all, alpha_pred.to("cpu")], dim=0)
    if alpha_only:
        return alpha_pred_all
    else:
        return orig_Y_all, X_duplicate_all, alpha_pred_all

orig_Y_all, X_duplicate_all, alpha_pred_all = compute_X_Y_alpha(model, full_loader)

In [6]:
# scores
corrects = (orig_Y_all.squeeze() == alpha_pred_all.max(-1)[1]).type(torch.DoubleTensor)
# print(corrects)
p=torch.nn.functional.normalize(alpha_pred_all, p=1, dim=-1)
scores_aleatoric = p.max(-1)[0].cpu().detach().numpy()
print('aleatoric score:',scores_aleatoric)
scores_epistemic = alpha_pred_all.max(-1)[0].cpu().detach().numpy()
print('epistemic score:',scores_epistemic)

aleatoric score: [0.25       0.99897187 0.99771863 ... 0.99688177 0.90225964 0.99745776]
epistemic score: [1.00000000e+00 1.05100199e+05 3.82447518e+04 ... 3.11658711e+04
 2.73575512e+03 3.06025324e+04]


In [10]:
# use maximum of alpha as threshold
# torch.concat(torch.max(alpha_pred_all, dim=1).values, grd_truth)
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
max_alpha = torch.max(alpha_pred_all, dim=1).values
output = [[],[],[]]
for threshold in np.linspace(500, 20000,50):
    pred = (torch.max(alpha_pred_all, dim=1).values < threshold).int()
    acc = balanced_accuracy_score(pred, grd_truth)
    acc2 = accuracy_score(pred, grd_truth)
    output[0].append(threshold)
    output[1].append(acc)
    output[2].append(acc2)
    print(f'threshold:{threshold}, balanced accuracy:{acc}, accuracy:{acc2}')

index = output[1].index(max(output[1]))
print("###")
print(f'threshold:{output[0][index]}, balanced accuracy:{output[1][index]}, accuracy:{output[2][index]}')

threshold:500.0, balanced accuracy:0.4977211332333004, accuracy:0.8244
threshold:897.9591836734694, balanced accuracy:0.49780594062442945, accuracy:0.8178
threshold:1295.9183673469388, balanced accuracy:0.49738391407057436, accuracy:0.8132
threshold:1693.8775510204082, balanced accuracy:0.4970659107016301, accuracy:0.8096
threshold:2091.8367346938776, balanced accuracy:0.49864480577435866, accuracy:0.805
threshold:2489.795918367347, balanced accuracy:0.4986538157213794, accuracy:0.798
threshold:2887.7551020408164, balanced accuracy:0.4982863513298296, accuracy:0.7866
threshold:3285.714285714286, balanced accuracy:0.5054878637459432, accuracy:0.7714
threshold:3683.673469387755, balanced accuracy:0.5065411984694496, accuracy:0.7658
threshold:4081.6326530612246, balanced accuracy:0.5070556368137332, accuracy:0.7656
threshold:4479.591836734694, balanced accuracy:0.5076104594252746, accuracy:0.7658
threshold:4877.551020408164, balanced accuracy:0.5080146698493495, accuracy:0.7646
threshold:

In [18]:
# to see the table
threshold  = output[0][index]
pred = (torch.max(alpha_pred_all, dim=1).values < threshold).int()
grd_truth

tensor([1, 0, 1,  ..., 0, 0, 0])

In [19]:
len(grd_truth)

57580

In [24]:
# use normalized alpha
p = torch.nn.functional.normalize(alpha_pred_all, p=1, dim=-1)
scores = p.max(-1)[0].cpu().detach().numpy()
output = [[],[],[]]
for threshold in np.linspace(0.8,0.99999,50):
    pred = (scores < threshold).astype(int)
    acc = balanced_accuracy_score(pred, grd_truth)
    accuracy_score(pred, grd_truth)
    output[0].append(threshold)
    output[1].append(acc)
    output[2].append(acc2)
    # print(f'threshold:{threshold}, balanced accuracy:{acc}, accuracy:{acc2}')

index = output[1].index(max(output[1]))
print(f'threshold:{output[0][index]}, balanced accuracy:{output[1][index]}, accuracy:{output[2][index]}')

threshold:0.9975853106212426, balanced accuracy:0.515490877497828, accuracy:0.8367315039944425


/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [8]:
grd_truth

tensor([0, 0, 0,  ..., 0, 0, 1])

In [29]:
# use sum of alpha - minimum
min_alpha = torch.min(alpha_pred_all, dim=1).values
scores = torch.sum(alpha_pred_all,dim=1)-4*min_alpha
output = [[],[],[]]
for threshold in np.linspace(2000,20000,50):
    pred = (scores < threshold).int()
    acc = balanced_accuracy_score(pred, grd_truth)
    accuracy_score(pred, grd_truth)
    output[0].append(threshold)
    output[1].append(acc)
    output[2].append(acc2)
    # print(f'threshold:{threshold}, balanced accuracy:{acc}, accuracy:{acc2}')
# print(pred)
index = output[1].index(max(output[1]))
print(f'threshold:{output[0][index]}, balanced accuracy:{output[1][index]}, accuracy:{output[2][index]}')

threshold:10448.979591836734, balanced accuracy:0.5156217144459242, accuracy:0.8367315039944425


/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/nfs/homedirs/zhz/anaconda3/envs/postnet/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2006: UserWarning: 